## 코드 설명  
  
웹크롤링, 스팀API Key, steam 라이브러리 이용
  
- input: 게임이름(영어) 리스트  
- output: 게임이름, 게임ID, 게임장르, 게임출시일 데이터셋  
  
#### URL = 'https://store.steampowered.com/search/results/?query&start='+str(start)+'&count=50&sort_by=Released_DESC&snr=1_7_7_popularnew_7&filter=popularnew&os=win&infinite=1'  

- URL 부분을 저희가 원하는 순서대로(인기순위) 바꾼다면 해당 코드 사용 가능  
  

### 인기게임 이름 가져오기

In [1]:
import requests
from bs4 import BeautifulSoup

**한국어 게임이름 리스트는 해당부분으로 수정하여 사용할 수 있으나 이후 정보를 가져올 수 없기 때문에 영어리스트 이용함**   
res = requests.get(URL, cookies={
        'Steam_Language':'koreana'
    })

In [2]:
result = []
rank_length = 400
# 50: 숨겨진 리스트가 50개씩 출력되기 때문
for start in range(0, rank_length, 50):
    URL = 'https://store.steampowered.com/search/results/?query&start='+str(start)+'&count=50&sort_by=Released_DESC&snr=1_7_7_popularnew_7&filter=popularnew&os=win&infinite=1'
    res = requests.get(URL)
    json_data = res.json()
    soup = BeautifulSoup(json_data['results_html'], 'html.parser')
    # steam = soup.select_one('#search_resultsRows')
    titles = soup.select('.col.search_name.ellipsis .title')
    
    for title in titles:
        result.append(title.text)


In [ ]:
result[:20]

### 데이터셋 가져오기

In [3]:
import os, json

from dotenv import load_dotenv
from steam import Steam
from datetime import datetime

from tqdm import tqdm

In [4]:
load_dotenv('.env')
KEY = os.getenv('STEAM_API_KEY')
steam = Steam(KEY)

In [6]:
%%time
gen_dict = {}
data_list = []
for game_name in tqdm(result[:50]):
    # 데이터 정보가 없는 경우도 있음
    try:
        # 게임ID 가져오기
        game_id = str(steam.apps.search_games(game_name)['apps'][0]['id'])
        # 게임정보 가져오기
        game_info = steam.apps.get_app_details(game_id)
        g_info = json.loads(game_info)
        # 해당 게임의 장르리스트 가져오기
        gens_list= []
        gens = g_info[game_id]['data']['genres']
        for gen in gens:
            gen_name = gen['description']
            gens_list.append(gen_name)
            
            # 전체 게임 장르 카운트
            key = gen['description']
            try:
                gen_dict[key] += 1
            except:
                gen_dict[key] = 1
            
        # 출시일 정보 가져오기
        g_date_str = g_info[game_id]['data']['release_date']['date']

        # date타입으로 변환
        g_date = datetime.strptime(g_date_str,'%d %b, %Y')

        # 우리가 원하는 date 시작, 끝 구간
        d1 = datetime(2018, 6, 1)
        d2 = datetime(2023, 6, 30)

        # 그 사이에 있는 날짜
        if (g_date-d1).days >0 and (g_date-d2).days < 0:
            # 자료
            record = (game_name, game_id, gens_list,g_date)
            
            data_list.append(record)
        
        
        
    except:
        pass

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [00:55<00:00,  1.11s/it]

CPU times: total: 4.31 s
Wall time: 55.7 s


In [8]:
len(data_list)

46

### 데이터셋으로 데이터프레임 만들기

In [9]:
import pandas as pd

In [11]:
column_name = ['game_name','game_id','game_genres','game_rel_d']

In [12]:
result_df = pd.DataFrame(data_list, columns = column_name)
result_df

,game_name,game_id,game_genres,game_rel_d
0,Ghost Trick: Phantom Detective,1967430,"[Action, Adventure]",2023-06-29
1,DAVE THE DIVER Digital Extra,2492320,"[Adventure, Casual, Indie, RPG, Simulation]",2023-06-28
2,DAVE THE DIVER,1868140,"[Adventure, Casual, Indie, RPG, Simulation]",2023-06-28
3,Swelter,1815330,[Action],2023-06-26
4,Brotato,1942280,"[Action, Casual, Indie, RPG]",2023-06-23
5,Nova Lands,1501610,"[Adventure, Indie, Simulation, Strategy]",2023-06-22
6,Six Days in Fallujah,1548850,"[Action, Early Access]",2023-06-22
7,Trepang2,1164940,"[Action, Indie]",2023-06-21
8,Let's School Homeroom,2431660,"[Casual, Indie, Simulation]",2023-06-20
9,Aliens: Dark Descent,1150440,"[Action, Strategy]",2023-06-19
